<a href="https://colab.research.google.com/github/shengyi2/spectrum_analysis/blob/main/Calculation_of_stoichiometry_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# Open absorbance spectra
  - FRET absorbance was previously process with the deconvolution script
  - file in csv (script included excel import function in case table is prepared elsewhere)
  - the table was prepared such that the columns are row number, wavelength, acceptor, donar, overlay, measured spectra

 - sample tables are shown below. 
 

|  | wavelength | acceptor | donar |overlay | measured spectra |
|---- | ---- | --- | --- | --- | ----|
|1 | 350 | 0.1 | 0.8 | 0.9 | 0.92 |
|2 | 355 | 0.2 | 0.6 | 0.8| 0.76|
|3 | 360 | 0.3 |0.4|0.7|0.72|

In [1]:
import pandas as pd

In [48]:
# open the file of deconvoluted spectra
def import_Colab(variable):
  print('Import',variable,'spectrum')
 
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    # read csv or excel
  if '.csv' in fn:
    import io
    data = pd.read_csv(io.BytesIO(uploaded[fn]))
    #data=pd.read_csv(fn,names=['wavelength',variable],dtype={'wavelength':int})
  elif '.xls' in fn:
    import io
    data = pd.read_excel(io.BytesIO(uploaded[fn]))
  else:
    print('File type not accepted, please rerun and select right file')
    return fn 
  return fn,data

In [3]:
  from google.colab import files
  

# import spectra on Colab

> Indented block




> 



In [481]:
# import FRET spectrum from local drive
global fn
fn=""
FRET_path,FRET_table =import_Colab('FRET')

Import FRET spectrum


Saving WELQ swg_processed.csv to WELQ swg_processed.csv
User uploaded file "WELQ swg_processed.csv" with length 45865 bytes


In [495]:
FRET_path

'WELQ swg_processed.csv'

# Lay out table for viewing and calculate stoichiometry

In [482]:
FRET_table.head()

,Unnamed: 0,wavelength,Acceptor,Donar,Overlay,Measured_curve
0,0,300,0.025233,0.048506,0.073739,0.062837
1,1,301,0.019615,0.041534,0.061149,0.049473
2,2,302,0.014919,0.035558,0.050477,0.038165
3,3,303,0.010959,0.030678,0.041636,0.027756
4,4,304,0.007643,0.025000,0.032644,0.019147


In [483]:
concat_table=FRET_table

In [484]:
import plotly.graph_objects as go



fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=concat_table.wavelength, y=concat_table.Donar,
                         mode='lines',
                         name='Donar'))
fig.add_trace(go.Scatter(x=concat_table.wavelength, y=concat_table.Acceptor,
                         mode='lines',
                         name='Acceptor'))
fig.add_trace(go.Scatter(x=concat_table.wavelength, y=concat_table.Overlay,
                         mode='lines',
                         name='Overlay'))
fig.add_trace(go.Scatter(x=concat_table.wavelength, y=concat_table.Measured_curve,
                         mode='lines',
                         name='Measured_curve'))

fig.show()

## Calculate the stoichiometry and save data

In [485]:
# Assign a table for results

In [486]:
gScarlet_EC=87600
mScarletI_EC=104000

In [487]:
Donar_max=FRET_table.Donar.max()

In [488]:
Acceptor_max=FRET_table.Acceptor.max()

In [489]:
Acceptor_max_wave=FRET_table.Acceptor.idxmax()

In [490]:
Donar_max_wave=FRET_table.Donar.idxmax()

In [491]:
if Acceptor_max_wave > Donar_max_wave:
  gScarlet_max=Donar_max
  gScarlet_max_wave=Donar_max_wave
  mScarlet_max=Acceptor_max
  mScarlet_max_wave=Acceptor_max_wave
else:
  gScarlet_max=Acceptor_max
  gScarlet_max_wave=Acceptor_max_wave
  mScarlet_max=Donar_max
  mScarlet_max_wave=Donar_max_wave



In [492]:
gScarlet_rel_num=gScarlet_max/gScarlet_EC
mScarlet_rel_num=mScarlet_max/mScarletI_EC
stoich=gScarlet_rel_num/mScarlet_rel_num

In [493]:
data1={'index':['EC','Max','Max_wavelength','relative amount','stoichiometry (green to red)'],
       'gScarlet':[gScarlet_EC,gScarlet_max,gScarlet_max_wave,gScarlet_rel_num,stoich],
      'mScarlet':[mScarletI_EC,mScarlet_max,mScarlet_max_wave,mScarlet_rel_num,stoich]}
result_table = pd.DataFrame(data=data1)
result_table

,index,gScarlet,mScarlet
0,EC,87600.000000,104000.000000
1,Max,0.679622,0.903792
2,Max_wavelength,197.000000,267.000000
3,relative amount,0.000008,0.000009
4,stoichiometry (green to red),0.892747,0.892747


In [494]:
file_name=FRET_path.replace('.csv','_stoichiometry.csv')
print(file_name)
result_table.to_csv(file_name) 

WELQ swg_processed_stoichiometry.csv
